In [1]:
import yfinance as yf
from pandas_datareader import data as pdr
import pandas as pd
import yahoo_fin.stock_info as si 
from yahoo_fin import *

W – means there are outstanding warrants.
R – means there is some kind of “rights” issue.
P – means “First Preferred Issue”. Preferred stocks are separate entity.
Q - means bankruptcy.

In [2]:
# deleting stock symbols/tickers within the nasdaq listing
nasdaq = pd.DataFrame(si.tickers_nasdaq())
sym = set( symbol for symbol in nasdaq[0].values.tolist() )

# deleting the inactive tickers by suffix
my_list = ['W', 'R', 'P', 'Q']

# variable for tickers to be saved and disgarded
del_set = set()
sav_set = set()

# loop to id the suffix and disgard
for symbol in sym:
    if len(symbol) > 4 and symbol[-1] in my_list:
        del_set.add(symbol)
    else:
        sav_set.add(symbol)

tickers = sorted(sav_set)

In [3]:
# populating a list for the latest movement list for each stock symbol
movementlist = []
for stock in tickers:
    # get history
    thestock = yf.download(tickers = stock, period = "5d", interval = "1d", ignore_tz = True, prepost = False)
    # hist = thestock.history(period="5d")
    # print(stock)
    low = float(10000)
    high = float(0)
    
    # print(thestock.info)
for day in thestock.itertuples(index=True, name='Pandas'):
    if day.Low < low:
        low = day.Low
    if high < day.High:
        high = day.High

    deltapercent = 100 * (high - low)/low
    Open = thestock(period = "5d", 0, "Open")
      # some error handling: 
    if len(hist >= 5):
        Close = thestock(period = "5d", 4, "Close")
    else:
        Close = Open
    if(Open == 0):
        deltaprice = 0
    else:
        deltaprice = 100 * (Close - Open) / Open
        print(stock + " " + str(deltapercent) + " " + str(deltaprice))
    
pair = [stock, deltapercent, deltaprice]
movementlist.append(pair)

SyntaxError: positional argument follows keyword argument (2290622105.py, line 19)

In [ ]:
for entry in movementlist:
    if entry[1]>float(100):
    print(entry)

In [ ]:
# High risers:
def lookup_stockinfo(thestock):
    try:
        return thestock.info
    except IndexError:
        return 0
        cutoff=float(80)
        
for entry in movementlist:
    if entry[2]>cutoff:
    print("\n"+ str(entry))
        thestock = yf.Ticker(str(entry[0]))
    if entry[0]=='HJLIW':
    print("no info")
    else:
        a = lookup_stockinfo(thestock)
    if a == 0:
        print("no info")
    else:
        if a is None:
            print("no info")
    else:
        if a == "":
            print("no")
        else:
            print(a)
            print('Up '+ str(entry[2]) + "%")
            print(str(a['sector']))
            print(str(a['longBusinessSummary']))
            print("year high "+ str(a['fiftyTwoWeekHigh']))

In [ ]:
movement = pd.DataFrame(entry)
movement

In [ ]:
# creating a csv file of the DataFrame
movement.to_csv('latest_movement.csv', header=True, index=True, encoding='utf-8')